# EECS 3401 Project 

## Filght Status Prediction


In [79]:
print("Hello World")

Hello World


In [80]:
import zipfile
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
#model evaluation form sklearn 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder


# one hot encoder from scikit learn
from sklearn.preprocessing import OneHotEncoder

# imputer from scikit learn
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [81]:
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [82]:
# Specify the path to your compressed folder and the CSV file inside
compressed_folder_path = 'Combined_Flights_2022.zip'
csv_file_name = 'Combined_Flights_2022.csv'

# Extract the contents of the ZIP file
with zipfile.ZipFile(compressed_folder_path, 'r') as zip_ref:
    zip_ref.extractall('path/to/extracted/folder')

# Create the full path to the extracted CSV file
csv_file_path = f'path/to/extracted/folder/{csv_file_name}'

# Read the CSV file using pandas
df = pd.read_csv(csv_file_path)
#df = pd.read_csv('Combined_Flights_2022.csv')
df.head(10)

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,...,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,...,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,...,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,GPT,False,False,1435,1430.0,0.0,-5.0,...,1446.0,1543.0,4.0,1605,-18.0,0.0,-2.0,1600-1659,2,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1135,1135.0,0.0,0.0,...,1154.0,1243.0,8.0,1245,6.0,0.0,0.0,1200-1259,2,0
5,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,TUL,False,False,955,952.0,0.0,-3.0,...,1017.0,1234.0,4.0,1240,-2.0,0.0,-1.0,1200-1259,3,0
6,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,LCH,False,False,2139,2136.0,0.0,-3.0,...,2147.0,2213.0,5.0,2231,-13.0,0.0,-1.0,2200-2259,1,0
7,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",TYS,IAH,False,False,1129,1117.0,0.0,-12.0,...,1139.0,1255.0,16.0,1306,5.0,0.0,0.0,1300-1359,4,0
8,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,AEX,False,False,1424,1414.0,0.0,-10.0,...,1430.0,1507.0,6.0,1524,-11.0,0.0,-1.0,1500-1559,1,0
9,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,MOB,False,False,954,947.0,0.0,-7.0,...,1004.0,1104.0,6.0,1121,-11.0,0.0,-1.0,1100-1159,2,0


In [83]:
columns = df.columns
len(columns)

61

In [84]:
columns

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

In [85]:
len(df.index)

4078318

In [86]:
origin_airtport_city = df['Origin'].unique()
origin_airtport_city

array(['GJT', 'HRL', 'DRO', 'IAH', 'DEN', 'TYS', 'MOB', 'LNK', 'BHM',
       'IAD', 'ECP', 'AEX', 'LFT', 'BTR', 'ABQ', 'BUF', 'ALB', 'HSV',
       'TUL', 'CPR', 'CRP', 'EWR', 'DCA', 'STL', 'CVG', 'RIC', 'PVD',
       'CLT', 'GSO', 'ERI', 'ORD', 'LEX', 'ABE', 'CAK', 'AVL', 'DAY',
       'CHO', 'ROA', 'PWM', 'LIT', 'MKE', 'BTV', 'OMA', 'GRB', 'MLI',
       'SDF', 'FNT', 'CHS', 'PIT', 'TVC', 'ATW', 'DLH', 'ROC', 'SPI',
       'HPN', 'MBS', 'CID', 'MYR', 'CAE', 'SGF', 'AVP', 'GRR', 'DSM',
       'CLE', 'PIA', 'MSN', 'MDT', 'SBN', 'MEM', 'ICT', 'SHV', 'XNA',
       'DIK', 'GPT', 'PNS', 'LRD', 'LCH', 'MFE', 'FLG', 'BRO', 'SAF',
       'PQI', 'ORF', 'LGA', 'SYR', 'RDU', 'ILM', 'IND', 'PHL', 'CMH',
       'SCE', 'ITH', 'JAX', 'OKC', 'JAN', 'GSP', 'TXK', 'ELP', 'MAF',
       'AUS', 'IDA', 'BNA', 'SAV', 'AMA', 'ATL', 'BDL', 'BLI', 'BOI',
       'BOS', 'BUR', 'BWI', 'BZN', 'COS', 'DAL', 'DTW', 'EUG', 'FAT',
       'FLL', 'GEG', 'HDN', 'HNL', 'HOU', 'ISP', 'ITO', 'KOA', 'LAS',
       'LAX', 'LBB',

In [87]:
busiest_airports = df['Origin'].value_counts().nlargest(10)
busiest_airports

Origin
ATL    183697
ORD    172169
DFW    161218
DEN    158092
CLT    126088
LAX    111935
LGA    100260
SEA     98884
LAS     97674
PHX     96095
Name: count, dtype: int64

In [88]:
specific_origin_airport = 'ATL'  # Replace with the airport code you're interested in
df_ATL_filtered = df[df['Origin'] == specific_origin_airport]
df_ATL_filtered.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
1855,2022-04-01,Southwest Airlines Co.,ATL,AUS,False,False,820,832.0,12.0,12.0,...,845.0,1000.0,4.0,1000,4.0,0.0,0.0,1000-1059,4,0
1856,2022-04-01,Southwest Airlines Co.,ATL,AUS,False,False,1440,1505.0,25.0,25.0,...,1516.0,1625.0,7.0,1615,17.0,1.0,1.0,1600-1659,4,0
1857,2022-04-01,Southwest Airlines Co.,ATL,AUS,False,False,1955,2105.0,70.0,70.0,...,2123.0,2232.0,5.0,2135,62.0,1.0,4.0,2100-2159,4,0
1858,2022-04-01,Southwest Airlines Co.,ATL,BNA,False,False,1510,1547.0,37.0,37.0,...,1601.0,1539.0,8.0,1510,37.0,1.0,2.0,1500-1559,1,0
1859,2022-04-01,Southwest Airlines Co.,ATL,BNA,False,False,820,819.0,0.0,-1.0,...,833.0,813.0,5.0,835,-17.0,0.0,-2.0,0800-0859,1,0


## Dropping Unwanted columns

These columns represent unique IDs and code which don't provide much value
in the training the model and are unrealated to the data analysis.

In [89]:
# List of columns to drop (IDs)
id_columns_to_drop = ['DOT_ID_Marketing_Airline', 'DOT_ID_Operating_Airline', 'OriginAirportID',
                      'OriginAirportSeqID', 'OriginCityMarketID', 'DestAirportID', 'DestAirportSeqID',
                      'DestCityMarketID', 'Flight_Number_Marketing_Airline', 'Flight_Number_Operating_Airline',
                      'IATA_Code_Marketing_Airline', 'IATA_Code_Operating_Airline', 'Tail_Number',
                      'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
                   'OriginCityName', 'OriginState', 'DestCityName', 'DestState',
                      'OriginStateName','OriginStateFips','OriginWac'
                  ,'DestStateName','DestStateFips','DestWac','Operating_Airline']

# Drop ID columns from the DataFrame
df_ATL_filtered = df_ATL_filtered.drop(columns=id_columns_to_drop)

# Print data types of the remaining columns
print(df_ATL_filtered.dtypes)

FlightDate               object
Airline                  object
Origin                   object
Dest                     object
Cancelled                  bool
Diverted                   bool
CRSDepTime                int64
DepTime                 float64
DepDelayMinutes         float64
DepDelay                float64
ArrTime                 float64
ArrDelayMinutes         float64
AirTime                 float64
CRSElapsedTime          float64
ActualElapsedTime       float64
Distance                float64
Year                      int64
Quarter                   int64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrDelay                float64
ArrDel15                float64
ArrivalD

## Pre Processing Data for model training 

### - Exploring the columns.

In [90]:
columns = df_ATL_filtered.columns
len(columns)

35

In [91]:
len(df_ATL_filtered.index)

183697

In [92]:
cancel_count = df_ATL_filtered['Cancelled'].value_counts()
cancel_count

Cancelled
False    180265
True       3432
Name: count, dtype: int64

In [93]:
airlines = df_ATL_filtered['Airline'].unique()
len(airlines)

14

In [94]:
dest = df_ATL_filtered['Dest'].unique()
len(dest)

154

In [95]:
df_ATL_filtered[['Airline','ArrDelay','DepDelay','ArrDelayMinutes','DepDelayMinutes','ArrTimeBlk','DepTimeBlk','FlightDate','Origin','Dest','DepartureDelayGroups','ArrivalDelayGroups']]

,Airline,ArrDelay,DepDelay,ArrDelayMinutes,DepDelayMinutes,ArrTimeBlk,DepTimeBlk,FlightDate,Origin,Dest,DepartureDelayGroups,ArrivalDelayGroups
1855,Southwest Airlines Co.,4.0,12.0,4.0,12.0,1000-1059,0800-0859,2022-04-01,ATL,AUS,0.0,0.0
1856,Southwest Airlines Co.,17.0,25.0,17.0,25.0,1600-1659,1400-1459,2022-04-01,ATL,AUS,1.0,1.0
1857,Southwest Airlines Co.,62.0,70.0,62.0,70.0,2100-2159,1900-1959,2022-04-01,ATL,AUS,4.0,4.0
1858,Southwest Airlines Co.,37.0,37.0,37.0,37.0,1500-1559,1500-1559,2022-04-01,ATL,BNA,2.0,2.0
1859,Southwest Airlines Co.,-17.0,-1.0,0.0,0.0,0800-0859,0800-0859,2022-04-01,ATL,BNA,-1.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4077926,Republic Airlines,-4.0,11.0,0.0,11.0,2100-2159,1900-1959,2022-03-31,ATL,IAD,0.0,-1.0
4077960,Republic Airlines,38.0,35.0,38.0,35.0,2200-2259,1900-1959,2022-03-31,ATL,EWR,2.0,2.0
4078230,Republic Airlines,NaN,29.0,NaN,29.0,2200-2259,1900-1959,2022-03-24,ATL,EWR,1.0,NaN
4078259,Republic Airlines,161.0,149.0,161.0,149.0,1900-1959,1700-1759,2022-03-13,ATL,EWR,9.0,10.0


We can observe follwing - 
1. There are 14 unique airlines taking off from the Atlante airport. This information won't be required in model training.
2. There are 154 destination where the origin was 'ATL'.
3. ArrTimeBlk and DepTimeBlk columns are represent hourly intervals so we will take in consideration the start of the hour
for example - '1000-1059' we will on consider '10' hours.
4. The columns representing _Delay and _DelayMinutes represent same things but have slight differnce in handling early departures and early arrivals
for our model we to check by using both columns separtly and compare which contributes to more accurate models.
5. Orgin column will be dropped cause its redundant for us.
6. We will also drop the year and quarter columns, the  main reason for that is we are taking data from year 2022 thats why its totally 
useless for us.
7. We need to convert the columns reperesenting time in 'hhmm' format using sine and cosine function to capture their cyclical nature,
if we don't do so then the ml algorithms will plot time as continous form which is not the case


### - Dropping the uneccesary columns

In [96]:
# dropping columns that are not useful for model training.
# Initially we will drop the ArrDelay and DepDelay for the first run
list_cols_drop_ml = ['FlightDate','Year','Airline','Origin','Dest','ArrDelay', 'DepDelay']
df_ATL_filtered = df_ATL_filtered.drop(columns=list_cols_drop_ml)
print(df_ATL_filtered.dtypes)



Cancelled                  bool
Diverted                   bool
CRSDepTime                int64
DepTime                 float64
DepDelayMinutes         float64
ArrTime                 float64
ArrDelayMinutes         float64
AirTime                 float64
CRSElapsedTime          float64
ActualElapsedTime       float64
Distance                float64
Quarter                   int64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrDel15                float64
ArrivalDelayGroups      float64
ArrTimeBlk               object
DistanceGroup             int64
DivAirportLandings        int64
dtype: object


In [97]:
columns = df_ATL_filtered.columns
print('Total columns - ' + str(len(columns)))

Total columns - 28


### - Extracting hour from the ArrTimeBlk and DepTimeBlk time intervals

In [98]:
# 'ArrTimeBlk' and 'DepTimeBlk' are the columns with time block intervals


df_ATL_filtered['ArrTimeBlk_numeric'] = df_ATL_filtered['ArrTimeBlk'].str.split('-').str[0].astype(int)

# Repeat the process for 'DepTimeBlk' 
df_ATL_filtered['DepTimeBlk_numeric'] = df_ATL_filtered['DepTimeBlk'].str.split('-').str[0].astype(int)

# Print data types of the updated DataFrame
print(df_ATL_filtered.dtypes)

Cancelled                  bool
Diverted                   bool
CRSDepTime                int64
DepTime                 float64
DepDelayMinutes         float64
ArrTime                 float64
ArrDelayMinutes         float64
AirTime                 float64
CRSElapsedTime          float64
ActualElapsedTime       float64
Distance                float64
Quarter                   int64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrDel15                float64
ArrivalDelayGroups      float64
ArrTimeBlk               object
DistanceGroup             int64
DivAirportLandings        int64
ArrTimeBlk_numeric        int32
DepTimeBlk_numeric        int32
dtype: object


In [99]:
# dropping the 'ArrTimeBlk' and DepTimeBlk
df_ATL_filtered = df_ATL_filtered.drop(columns=['DepTimeBlk', 'ArrTimeBlk'])
df_ATL_filtered.head(5)

,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDel15,ArrivalDelayGroups,DistanceGroup,DivAirportLandings,ArrTimeBlk_numeric,DepTimeBlk_numeric
1855,False,False,820,832.0,12.0,1004.0,4.0,135.0,160.0,152.0,...,845.0,1000.0,4.0,1000,0.0,0.0,4,0,1000,800
1856,False,False,1440,1505.0,25.0,1632.0,17.0,129.0,155.0,147.0,...,1516.0,1625.0,7.0,1615,1.0,1.0,4,0,1600,1400
1857,False,False,1955,2105.0,70.0,2237.0,62.0,129.0,160.0,152.0,...,2123.0,2232.0,5.0,2135,1.0,4.0,4,0,2100,1900
1858,False,False,1510,1547.0,37.0,1547.0,37.0,38.0,60.0,60.0,...,1601.0,1539.0,8.0,1510,1.0,2.0,1,0,1500,1500
1859,False,False,820,819.0,0.0,818.0,0.0,40.0,75.0,59.0,...,833.0,813.0,5.0,835,0.0,-2.0,1,0,800,800


### - Handling columns with value in 'hhmm' format

In [100]:
# Columns that are in the "hhmm" format
df_ATL_filtered[['CRSDepTime','DepTime','CRSArrTime','ArrTime','WheelsOff','WheelsOn','ArrTimeBlk_numeric','DepTimeBlk_numeric']]


,CRSDepTime,DepTime,CRSArrTime,ArrTime,WheelsOff,WheelsOn,ArrTimeBlk_numeric,DepTimeBlk_numeric
1855,820,832.0,1000,1004.0,845.0,1000.0,1000,800
1856,1440,1505.0,1615,1632.0,1516.0,1625.0,1600,1400
1857,1955,2105.0,2135,2237.0,2123.0,2232.0,2100,1900
1858,1510,1547.0,1510,1547.0,1601.0,1539.0,1500,1500
1859,820,819.0,835,818.0,833.0,813.0,800,800
...,...,...,...,...,...,...,...,...
4077926,1929,1940.0,2120,2116.0,1957.0,2108.0,2100,1900
4077960,1952,2027.0,2220,2258.0,2050.0,2221.0,2200,1900
4078230,1947,2016.0,2200,42.0,2105.0,23.0,2200,1900
4078259,1704,1933.0,1918,2159.0,1955.0,2145.0,1900,1700


In [101]:
time_columns = ['CRSDepTime', 'DepTime', 'CRSArrTime', 'ArrTime', 'WheelsOff', 'WheelsOn', 'ArrTimeBlk_numeric', 'DepTimeBlk_numeric']

for column in time_columns:
    # Extract hour and minute
    df_ATL_filtered[column + '_hour'] = df_ATL_filtered[column] // 100
    df_ATL_filtered[column + '_minute'] = df_ATL_filtered[column] % 100

    # Apply trigonometric transformations
    df_ATL_filtered[column + '_sin'] = np.sin(2 * np.pi * (df_ATL_filtered[column + '_hour'] * 60 + df_ATL_filtered[column + '_minute']) / (24 * 60))
    df_ATL_filtered[column + '_cos'] = np.cos(2 * np.pi * (df_ATL_filtered[column + '_hour'] * 60 + df_ATL_filtered[column + '_minute']) / (24 * 60))

    # Drop the original columns
    df_ATL_filtered = df_ATL_filtered.drop(columns=[column, column + '_hour', column + '_minute'])

# Print data types of the updated DataFrame
    print(df_ATL_filtered.dtypes)
    
  

Cancelled                  bool
Diverted                   bool
DepTime                 float64
DepDelayMinutes         float64
ArrTime                 float64
ArrDelayMinutes         float64
AirTime                 float64
CRSElapsedTime          float64
ActualElapsedTime       float64
Distance                float64
Quarter                   int64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrDel15                float64
ArrivalDelayGroups      float64
DistanceGroup             int64
DivAirportLandings        int64
ArrTimeBlk_numeric        int32
DepTimeBlk_numeric        int32
CRSDepTime_sin          float64
CRSDepTime_cos          float64
dtype: object
Cancelled                  bool
Diverted                  

Cancelled                  bool
Diverted                   bool
DepDelayMinutes         float64
ArrTime                 float64
ArrDelayMinutes         float64
AirTime                 float64
CRSElapsedTime          float64
ActualElapsedTime       float64
Distance                float64
Quarter                   int64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
ArrDel15                float64
ArrivalDelayGroups      float64
DistanceGroup             int64
DivAirportLandings        int64
ArrTimeBlk_numeric        int32
DepTimeBlk_numeric        int32
CRSDepTime_sin          float64
CRSDepTime_cos          float64
DepTime_sin             float64
DepTime_cos             float64
CRSArrTime_sin          float64
CRSArrTime_cos          float64
dtype: o

##### Explaination -

1. **Converting 'hhmm' to Minutes:**
   - `df_ATL_filtered[column + '_hour'] * 60 + df_ATL_filtered[column + '_minute']`: This part converts the 'hhmm' format to minutes. It takes the hour component, multiplies it by 60 to convert to minutes, and adds the minute component.

2. **Normalizing to [0, 1):**
   - `(df_ATL_filtered[column + '_hour'] * 60 + df_ATL_filtered[column + '_minute']) / (24 * 60)`: This part normalizes the minutes to the range [0, 1) by dividing by the total number of minutes in a day (24 hours * 60 minutes).

3. **Trigonometric Transformations:**
   - `2 * np.pi * ...`: This part scales the normalized time to the range [0, 2π), making it suitable for trigonometric functions.

   - `np.sin(...)`: This calculates the sine of the scaled, normalized time. The sine function produces a wave-like pattern with values between -1 and 1. It captures the cyclical nature of time, where similar times of day will have similar sine values.

   - `np.cos(...)`: Similarly, this calculates the cosine of the scaled, normalized time. The cosine function is another periodic function that is shifted relative to the sine function. It provides an additional perspective on the cyclical pattern.

In summary, the sine and cosine transformations encode the time information into two continuous features that capture both the cyclicality and the phase (shift) within the day. This encoding is often beneficial for machine learning models to capture the temporal patterns in a way that respects the circular nature of time. The resulting features (`column + '_sin'` and `column + '_cos'`) can be used as inputs to your machine learning model for predicting flight cancellations.

### - Converting columns with Boolean values to Binary

In [102]:
df_ATL_filtered['Cancelled_binary'] = df_ATL_filtered['Cancelled'].astype(int)
df_ATL_filtered['Diverted_binary'] = df_ATL_filtered['Diverted'].astype(int)

# Drop the original boolean columns if needed
df_ATL_filtered = df_ATL_filtered.drop(columns=['Cancelled', 'Diverted'])

# Print data types of the updated DataFrame
print(df_ATL_filtered.dtypes)

DepDelayMinutes           float64
ArrDelayMinutes           float64
AirTime                   float64
CRSElapsedTime            float64
ActualElapsedTime         float64
Distance                  float64
Quarter                     int64
Month                       int64
DayofMonth                  int64
DayOfWeek                   int64
DepDel15                  float64
DepartureDelayGroups      float64
TaxiOut                   float64
TaxiIn                    float64
ArrDel15                  float64
ArrivalDelayGroups        float64
DistanceGroup               int64
DivAirportLandings          int64
CRSDepTime_sin            float64
CRSDepTime_cos            float64
DepTime_sin               float64
DepTime_cos               float64
CRSArrTime_sin            float64
CRSArrTime_cos            float64
ArrTime_sin               float64
ArrTime_cos               float64
WheelsOff_sin             float64
WheelsOff_cos             float64
WheelsOn_sin              float64
WheelsOn_cos  

In [103]:
df_ATL_filtered.head(5)

,DepDelayMinutes,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Quarter,Month,DayofMonth,DayOfWeek,...,WheelsOff_sin,WheelsOff_cos,WheelsOn_sin,WheelsOn_cos,ArrTimeBlk_numeric_sin,ArrTimeBlk_numeric_cos,DepTimeBlk_numeric_sin,DepTimeBlk_numeric_cos,Cancelled_binary,Diverted_binary
1855,12.0,4.0,135.0,160.0,152.0,813.0,2,4,1,5,...,0.751840,-0.659346,0.500000,-0.866025,0.500000,-0.866025,0.866025,-0.500000,0,0
1856,25.0,17.0,129.0,155.0,147.0,813.0,2,4,1,5,...,-0.754710,-0.656059,-0.915311,-0.402747,-0.866025,-0.500000,-0.500000,-0.866025,0,0
1857,70.0,62.0,129.0,160.0,152.0,813.0,2,4,1,5,...,-0.632705,0.774393,-0.374607,0.927184,-0.707107,0.707107,-0.965926,0.258819,0,0
1858,37.0,37.0,38.0,60.0,60.0,214.0,2,4,1,5,...,-0.868199,-0.496217,-0.816642,-0.577145,-0.707107,-0.707107,-0.707107,-0.707107,0,0
1859,0.0,0.0,40.0,75.0,59.0,214.0,2,4,1,5,...,0.785317,-0.619094,0.836286,-0.548293,0.866025,-0.500000,0.866025,-0.500000,0,0


### - Handling the misssing values in the dataset

In [104]:
missing_values = df_ATL_filtered.isnull().sum()
    
# Display missing values count for each column
print("Missing Values Count for Each Column:")
print(missing_values)


Missing Values Count for Each Column:
DepDelayMinutes           3388
ArrDelayMinutes           3845
AirTime                   3845
CRSElapsedTime               0
ActualElapsedTime         3845
Distance                     0
Quarter                      0
Month                        0
DayofMonth                   0
DayOfWeek                    0
DepDel15                  3388
DepartureDelayGroups      3388
TaxiOut                   3425
TaxiIn                    3453
ArrDel15                  3845
ArrivalDelayGroups        3845
DistanceGroup                0
DivAirportLandings           0
CRSDepTime_sin               0
CRSDepTime_cos               0
DepTime_sin               3388
DepTime_cos               3388
CRSArrTime_sin               0
CRSArrTime_cos               0
ArrTime_sin               3453
ArrTime_cos               3453
WheelsOff_sin             3425
WheelsOff_cos             3425
WheelsOn_sin              3453
WheelsOn_cos              3453
ArrTimeBlk_numeric_sin       0
A

In [105]:
df_ATL_filtered.dropna()


,DepDelayMinutes,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Quarter,Month,DayofMonth,DayOfWeek,...,WheelsOff_sin,WheelsOff_cos,WheelsOn_sin,WheelsOn_cos,ArrTimeBlk_numeric_sin,ArrTimeBlk_numeric_cos,DepTimeBlk_numeric_sin,DepTimeBlk_numeric_cos,Cancelled_binary,Diverted_binary
1855,12.0,4.0,135.0,160.0,152.0,813.0,2,4,1,5,...,0.751840,-0.659346,0.500000,-0.866025,0.500000,-0.866025,8.660254e-01,-0.500000,0,0
1856,25.0,17.0,129.0,155.0,147.0,813.0,2,4,1,5,...,-0.754710,-0.656059,-0.915311,-0.402747,-0.866025,-0.500000,-5.000000e-01,-0.866025,0,0
1857,70.0,62.0,129.0,160.0,152.0,813.0,2,4,1,5,...,-0.632705,0.774393,-0.374607,0.927184,-0.707107,0.707107,-9.659258e-01,0.258819,0,0
1858,37.0,37.0,38.0,60.0,60.0,214.0,2,4,1,5,...,-0.868199,-0.496217,-0.816642,-0.577145,-0.707107,-0.707107,-7.071068e-01,-0.707107,0,0
1859,0.0,0.0,40.0,75.0,59.0,214.0,2,4,1,5,...,0.785317,-0.619094,0.836286,-0.548293,0.866025,-0.500000,8.660254e-01,-0.500000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4077918,6.0,0.0,99.0,140.0,129.0,746.0,1,3,31,4,...,0.496217,-0.868199,0.087156,-0.996195,0.258819,-0.965926,7.071068e-01,-0.707107,0,0
4077926,11.0,0.0,71.0,111.0,96.0,534.0,1,3,31,4,...,-0.872496,0.488621,-0.681998,0.731354,-0.707107,0.707107,-9.659258e-01,0.258819,0,0
4077960,35.0,38.0,91.0,148.0,151.0,746.0,1,3,31,4,...,-0.737277,0.675590,-0.418660,0.908143,-0.500000,0.866025,-9.659258e-01,0.258819,0,0
4078259,149.0,161.0,110.0,134.0,146.0,746.0,1,3,13,7,...,-0.876727,0.480989,-0.555570,0.831470,-0.965926,0.258819,-9.659258e-01,-0.258819,0,0


In [106]:
df_ATL_filtered.corr()

,DepDelayMinutes,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Quarter,Month,DayofMonth,DayOfWeek,...,WheelsOff_sin,WheelsOff_cos,WheelsOn_sin,WheelsOn_cos,ArrTimeBlk_numeric_sin,ArrTimeBlk_numeric_cos,DepTimeBlk_numeric_sin,DepTimeBlk_numeric_cos,Cancelled_binary,Diverted_binary
DepDelayMinutes,1.000000,0.980331,0.011742,0.013093,0.019081,0.012011,0.053485,0.062578,-0.003116,0.024621,...,-0.038381,0.105989,-0.006676,0.107216,-0.068015,0.102591,-0.090272,0.084653,0.015054,0.028870
ArrDelayMinutes,0.980331,1.000000,0.010283,0.001783,0.035799,0.001845,0.050734,0.056724,-0.006720,0.022340,...,-0.035247,0.096760,-0.007311,0.099981,-0.065529,0.093542,-0.084404,0.075964,NaN,NaN
AirTime,0.011742,0.010283,1.000000,0.991937,0.989773,0.989906,-0.008391,-0.010293,-0.004088,0.003371,...,0.035440,-0.043384,-0.008945,-0.011730,-0.006871,-0.011168,0.048294,-0.042376,NaN,NaN
CRSElapsedTime,0.013093,0.001783,0.991937,1.000000,0.983773,0.991919,-0.005318,-0.006199,-0.002275,0.004840,...,0.040640,-0.039268,-0.000839,-0.010187,-0.001822,-0.007538,0.050720,-0.037019,0.001634,0.003364
ActualElapsedTime,0.019081,0.035799,0.989773,0.983773,1.000000,0.979176,-0.003185,-0.007328,-0.006664,0.006178,...,0.037132,-0.038850,-0.006300,-0.007191,-0.004829,-0.008147,0.049438,-0.038818,NaN,NaN
Distance,0.012011,0.001845,0.989906,0.991919,0.979176,1.000000,0.010079,0.009745,-0.000639,0.005096,...,0.038647,-0.039188,-0.006066,-0.007723,-0.006658,-0.005596,0.049126,-0.037085,0.000119,0.002769
Quarter,0.053485,0.050734,-0.008391,-0.005318,-0.003185,0.010079,1.000000,0.917569,0.017404,0.032444,...,0.012205,0.022842,0.023643,0.018130,0.015356,0.015514,0.002021,0.022629,-0.021492,0.001475
Month,0.062578,0.056724,-0.010293,-0.006199,-0.007328,0.009745,0.917569,1.000000,0.015468,0.019178,...,0.014424,0.026314,0.026970,0.020648,0.016794,0.017470,0.002763,0.025360,-0.025825,0.004713
DayofMonth,-0.003116,-0.006720,-0.004088,-0.002275,-0.006664,-0.000639,0.017404,0.015468,1.000000,-0.008196,...,-0.000449,0.004300,0.001245,0.003601,-0.000441,0.004393,-0.002304,0.005132,-0.007333,0.000271
DayOfWeek,0.024621,0.022340,0.003371,0.004840,0.006178,0.005096,0.032444,0.019178,-0.008196,1.000000,...,-0.008039,0.003689,-0.007997,0.005586,-0.007129,0.001587,-0.005642,-0.000581,0.054500,0.002863


In [107]:
len(df_ATL_filtered.index)

183697

In [108]:
time_columns_sincos = ['CRSDepTime_sin','CRSDepTime_cos', 'DepTime_sin','DepTime_cos', 'CRSArrTime_sin','CRSArrTime_cos','ArrTime_sin','ArrTime_cos' ,'WheelsOff_sin', 'WheelsOff_cos','WheelsOn_cos','WheelsOn_sin', 'ArrTimeBlk_numeric_sin','ArrTimeBlk_numeric_cos', 'DepTimeBlk_numeric_sin','DepTimeBlk_numeric_cos']

#Replacing the Nan values with mean to tranform data that will be meaningful while finding the corr.

df_ATL_filtered['CRSDepTime_sin'] = df_ATL_filtered['CRSDepTime_sin'].fillna(df_ATL_filtered['CRSDepTime_sin'].mean)
df_ATL_filtered['CRSDepTime_sin'].isnull().sum()
df_ATL_filtered['CRSDepTime_cos'] = df_ATL_filtered['CRSDepTime_cos'].fillna(df_ATL_filtered['CRSDepTime_cos'].mean)
df_ATL_filtered['CRSDepTime_cos'].isnull().sum()
df_ATL_filtered['DepTime_sin'] = df_ATL_filtered['DepTime_sin'].fillna(df_ATL_filtered['DepTime_sin'].mean)
df_ATL_filtered['DepTime_sin'].isnull().sum()
df_ATL_filtered['DepTime_cos'] = df_ATL_filtered['DepTime_cos'].fillna(df_ATL_filtered['DepTime_cos'].mean)
df_ATL_filtered['DepTime_cos'].isnull().sum()
df_ATL_filtered['CRSArrTime_sin'] = df_ATL_filtered['CRSArrTime_sin'].fillna(df_ATL_filtered['CRSArrTime_sin'].mean)
df_ATL_filtered['CRSArrTime_sin'].isnull().sum()
df_ATL_filtered['CRSArrTime_cos'] = df_ATL_filtered['CRSArrTime_cos'].fillna(df_ATL_filtered['CRSArrTime_cos'].mean)
df_ATL_filtered['CRSArrTime_cos'].isnull().sum()
df_ATL_filtered['ArrTime_sin'] = df_ATL_filtered['ArrTime_sin'].fillna(df_ATL_filtered['ArrTime_sin'].mean)
df_ATL_filtered['ArrTime_sin'].isnull().sum()
df_ATL_filtered['ArrTime_cos'] = df_ATL_filtered['ArrTime_cos'].fillna(df_ATL_filtered['ArrTime_cos'].mean)
df_ATL_filtered['ArrTime_cos'].isnull().sum()
df_ATL_filtered['WheelsOff_sin'] = df_ATL_filtered['WheelsOff_sin'].fillna(df_ATL_filtered['WheelsOff_sin'].mean)
df_ATL_filtered['WheelsOff_sin'].isnull().sum()
df_ATL_filtered['WheelsOff_cos'] = df_ATL_filtered['WheelsOff_cos'].fillna(df_ATL_filtered['WheelsOff_cos'].mean)
df_ATL_filtered['WheelsOff_cos'].isnull().sum()
df_ATL_filtered['WheelsOn_sin'] = df_ATL_filtered['WheelsOn_sin'].fillna(df_ATL_filtered['WheelsOn_sin'].mean)
df_ATL_filtered['WheelsOn_sin'].isnull().sum()
df_ATL_filtered['WheelsOn_cos'] = df_ATL_filtered['WheelsOn_cos'].fillna(df_ATL_filtered['WheelsOn_cos'].mean)
df_ATL_filtered['WheelsOn_cos'].isnull().sum()

df_ATL_filtered['ArrTimeBlk_numeric_sin'] = df_ATL_filtered['ArrTimeBlk_numeric_sin'].fillna(df_ATL_filtered['ArrTimeBlk_numeric_sin'].mean)
df_ATL_filtered['ArrTimeBlk_numeric_sin'].isnull().sum()
df_ATL_filtered['ArrTimeBlk_numeric_cos'] = df_ATL_filtered['ArrTimeBlk_numeric_cos'].fillna(df_ATL_filtered['ArrTimeBlk_numeric_cos'].mean)
df_ATL_filtered['ArrTimeBlk_numeric_cos'].isnull().sum()

df_ATL_filtered['DepTimeBlk_numeric_sin'] = df_ATL_filtered['DepTimeBlk_numeric_sin'].fillna(df_ATL_filtered['DepTimeBlk_numeric_sin'].mean)
df_ATL_filtered['DepTimeBlk_numeric_sin'].isnull().sum()
df_ATL_filtered['DepTimeBlk_numeric_cos'] = df_ATL_filtered['DepTimeBlk_numeric_cos'].fillna(df_ATL_filtered['DepTimeBlk_numeric_cos'].mean)
df_ATL_filtered['DepTimeBlk_numeric_cos'].isnull().sum()



0

In [109]:
print(df_ATL_filtered['Cancelled_binary'])
df_ATL_filtered['Cancelled_binary'] = df_ATL_filtered['Cancelled_binary'].fillna(df_ATL_filtered['Cancelled_binary'].mode)
df_ATL_filtered['Cancelled_binary'].isnull().sum()

1855       0
1856       0
1857       0
1858       0
1859       0
          ..
4077926    0
4077960    0
4078230    0
4078259    0
4078304    0
Name: Cancelled_binary, Length: 183697, dtype: int32


0

In [110]:
print(df_ATL_filtered['Diverted_binary'])
df_ATL_filtered['Diverted_binary'] = df_ATL_filtered['Diverted_binary'].fillna(df_ATL_filtered['Diverted_binary'].mode)
df_ATL_filtered['Diverted_binary'].isnull().sum()

1855       0
1856       0
1857       0
1858       0
1859       0
          ..
4077926    0
4077960    0
4078230    1
4078259    0
4078304    0
Name: Diverted_binary, Length: 183697, dtype: int32


0

In [111]:
df_ATL_filtered.isnull().sum()

DepDelayMinutes           3388
ArrDelayMinutes           3845
AirTime                   3845
CRSElapsedTime               0
ActualElapsedTime         3845
Distance                     0
Quarter                      0
Month                        0
DayofMonth                   0
DayOfWeek                    0
DepDel15                  3388
DepartureDelayGroups      3388
TaxiOut                   3425
TaxiIn                    3453
ArrDel15                  3845
ArrivalDelayGroups        3845
DistanceGroup                0
DivAirportLandings           0
CRSDepTime_sin               0
CRSDepTime_cos               0
DepTime_sin                  0
DepTime_cos                  0
CRSArrTime_sin               0
CRSArrTime_cos               0
ArrTime_sin                  0
ArrTime_cos                  0
WheelsOff_sin                0
WheelsOff_cos                0
WheelsOn_sin                 0
WheelsOn_cos                 0
ArrTimeBlk_numeric_sin       0
ArrTimeBlk_numeric_cos       0
DepTimeB

### - Transforming the columns to train the model

We will predict the status of the flight. i.e. 'Cancelled_Vinary'.

In [112]:
df_ATL_filtered.head(5)

,DepDelayMinutes,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Quarter,Month,DayofMonth,DayOfWeek,...,WheelsOff_sin,WheelsOff_cos,WheelsOn_sin,WheelsOn_cos,ArrTimeBlk_numeric_sin,ArrTimeBlk_numeric_cos,DepTimeBlk_numeric_sin,DepTimeBlk_numeric_cos,Cancelled_binary,Diverted_binary
1855,12.0,4.0,135.0,160.0,152.0,813.0,2,4,1,5,...,0.75184,-0.659346,0.5,-0.866025,0.500000,-0.866025,0.866025,-0.500000,0,0
1856,25.0,17.0,129.0,155.0,147.0,813.0,2,4,1,5,...,-0.75471,-0.656059,-0.915311,-0.402747,-0.866025,-0.500000,-0.500000,-0.866025,0,0
1857,70.0,62.0,129.0,160.0,152.0,813.0,2,4,1,5,...,-0.632705,0.774393,-0.374607,0.927184,-0.707107,0.707107,-0.965926,0.258819,0,0
1858,37.0,37.0,38.0,60.0,60.0,214.0,2,4,1,5,...,-0.868199,-0.496217,-0.816642,-0.577145,-0.707107,-0.707107,-0.707107,-0.707107,0,0
1859,0.0,0.0,40.0,75.0,59.0,214.0,2,4,1,5,...,0.785317,-0.619094,0.836286,-0.548293,0.866025,-0.500000,0.866025,-0.500000,0,0


In [113]:
X = df_ATL_filtered.drop(['Cancelled_binary'],axis=1)
y = df_ATL_filtered['Cancelled_binary']
X_test,X_train,y_test,y_train = train_test_split(X,y,test_size =0.2)

In [114]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=150),
     "SVM": SVC()
}

def fit_and_score(models, X_train, X_test, y_train, y_test):
    '''
    Fits and evaluates given machine learning models.
    models : a dict of different Scikit-Learn machine learning models
    :param models: 
    :param X_train: 
    :param X_test: 
    :param y_train: 
    :param y_test: 
    :return: 
    '''
    
    #set the random seed 
    np.random.seed(42)
    
    # making dictionary to keep model scores 
    model_scores = {}
    # making dictionary to keep confusion matrices
    confusion_matrices = {}
    # making dictionary to keep classification reports
    classification_reports = {}
    
    # looping through the model
    for name, model in models.items():
        # Fitting the model to the data
        model.fit(X_train,y_train)
        # Predicting on the test set
        y_pred = model.predict(X_test)
        # evaluating the model and append its score to model_scores
        model_scores[name] = model.score(X_test,y_test)
        # Computing the confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        confusion_matrices[name] = cm
        # Generating the classification report
        report = classification_report(y_test, y_pred)
        classification_reports[name] = report
        
    return model_scores, confusion_matrices, classification_reports

In [115]:
model_scores, confusion_matrices, classification_reports = fit_and_score(models, X_train, X_test, y_train, y_test)
# printing model scores
model_scores

TypeError: float() argument must be a string or a real number, not 'method'